In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
from sklearn import metrics


from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, roc_auc_score,precision_score
from sklearn.model_selection import StratifiedKFold 
# evaluate label propagation on the semi-supervised learning dataset

dataSet = sp.loadmat('breastw.mat')
X = dataSet['X']
y = dataSet['y']
Covariance=['spherical', 'tied', 'diag', 'full']
Params=['kmeans', 'random']
skf = StratifiedKFold(n_splits=4) 
data = np.array([['Covariance','Paramêtre d initialisation','F1 score','recall','precision','accuracy','AUC','numero_fold']])
fold=0

for train_index, test_index in skf.split(X, y):
        fold=fold+1
        X_train, X_test = X[train_index,:], X[test_index,:]
        y_train, y_test = y[train_index], y[test_index]
        for i in Covariance:
                for j in Params:
                        
                        gmm = GaussianMixture(n_components=2,init_params=j,covariance_type=i)
                        gmm.fit(X_train,y_train)
                        y_pred=gmm.predict(X_test)
                        if precision_score(y_test,y_pred)< 0.15:
    
                                y_pred = np.array([0 if label == 1 else 1 for label in y_pred])
                                probas = 1-gmm.predict_proba(X_test)
                                probas = probas[:, 1]
                        F1=round(f1_score(y_test,y_pred),3)
                        recall=round(recall_score(y_test,y_pred),3)
                        precision=round(precision_score(y_test,y_pred,zero_division=0),3)
                        accuracy=round(accuracy_score(y_test,y_pred),3)
                        fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred)
                        roc_auc = metrics.auc(fpr, tpr)
                        data = np.append(data, [[i,j,F1,recall,precision,accuracy,roc_auc,fold]],axis=0)
                        


In [59]:
import pandas as pd

df = pd.DataFrame(data=data)
df

,0,1,2,3,4,5,6,7
0,Covariance,Paramêtre d initialisation,F1 score,recall,precision,accuracy,AUC,numero_fold
1,spherical,kmeans,0.909,1.0,0.833,0.93,0.9459459459459459,1
2,spherical,random,0.909,1.0,0.833,0.93,0.9459459459459459,1
3,tied,kmeans,0.836,0.767,0.92,0.895,0.8653153153153154,1
4,tied,random,0.459,0.567,0.386,0.532,0.5400900900900901,1
5,diag,kmeans,0.916,1.0,0.845,0.936,0.9504504504504505,1
6,diag,random,0.916,1.0,0.845,0.936,0.9504504504504505,1
7,full,kmeans,0.8,1.0,0.667,0.825,0.8648648648648649,1
8,full,random,0.8,1.0,0.667,0.825,0.8648648648648649,1
9,spherical,kmeans,0.894,0.983,0.819,0.918,0.9331081081081081,2
